<a href="https://colab.research.google.com/github/Girish-Kumar-Sahu/SmartCrop/blob/main/smartcrop_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
import gradio as gr
import numpy as np

# Load dataset
df = pd.read_csv("Crop_dataset.csv")

X = df[['N','P','K','temperature','humidity','ph','rainfall']]
y = df['label']

le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Train Accuracy:", model.score(X_train, y_train))
print("Test Accuracy:", model.score(X_test, y_test))

joblib.dump(model, "crop_model.joblib")
joblib.dump(le, "label_encoder.joblib")

model = joblib.load("crop_model.joblib")
le = joblib.load("label_encoder.joblib")

def recommend_crop(N, P, K, temperature, humidity, ph, rainfall):
    features = np.array([[N, P, K, temperature, humidity, ph, rainfall]])
    pred = model.predict(features)[0]
    crop = le.inverse_transform([pred])[0]

    yield_est = round(1000 + (rainfall - 50) * 5, 1)
    profit_est = round(yield_est * 2.5, 1)
    sustainability = max(0, 100 - abs(ph - 6.5) * 10)

    return {
        "Recommended Crop": crop,
        "Estimated Yield (kg/acre)": yield_est,
        "Estimated Profit (₹)": profit_est,
        "Sustainability Score": sustainability
    }

demo = gr.Interface(
    fn=recommend_crop,
    inputs=[
        gr.Number(label="Nitrogen (N)"),
        gr.Number(label="Phosphorus (P)"),
        gr.Number(label="Potassium (K)"),
        gr.Number(label="Temperature (°C)"),
        gr.Number(label="Humidity (%)"),
        gr.Number(label="Soil pH"),
        gr.Number(label="Rainfall (mm)")
    ],
    outputs="json",
    title="SmartCrop",
    description="AI-based Crop Recommendation System"
)

demo.launch(share=True)


Train Accuracy: 1.0
Test Accuracy: 0.9931818181818182
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7e0571cc8737deb7c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# New section